In [1]:
!wget https://he-s3.s3.amazonaws.com/media/hackathon/hackerearth-test-draft-1-102/predicting-tweet-sentiments-231101b4/fa62f5d69a9f11ea.zip

--2020-06-10 17:28:11--  https://he-s3.s3.amazonaws.com/media/hackathon/hackerearth-test-draft-1-102/predicting-tweet-sentiments-231101b4/fa62f5d69a9f11ea.zip
Resolving he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)... 52.219.124.8
Connecting to he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)|52.219.124.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 510744 (499K) [application/zip]
Saving to: ‘fa62f5d69a9f11ea.zip’

fa62f5d69a9f11ea.zi 100%[===================>] 498.77K   451KB/s    in 1.1s    

2020-06-10 17:28:13 (451 KB/s) - ‘fa62f5d69a9f11ea.zip’ saved [510744/510744]



In [2]:
!unzip fa62f5d69a9f11ea.zip

Archive:  fa62f5d69a9f11ea.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/test.csv        


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

In [5]:
train.isnull().sum(),test.isnull().sum(),train.shape,test.shape,train.dtypes

(id                 0
 original_text      0
 lang               4
 retweet_count      4
 original_author    0
 sentiment_class    0
 dtype: int64, id                 0
 original_text      0
 lang               0
 retweet_count      1
 original_author    0
 dtype: int64, (3235, 6), (1387, 5), id                 float64
 original_text       object
 lang                object
 retweet_count       object
 original_author     object
 sentiment_class      int64
 dtype: object)

In [0]:
train['lang']=train['lang'].fillna(method='bfill')
train['retweet_count']=train['retweet_count'].fillna(method='bfill')
test['retweet_count']=test['retweet_count'].fillna(method='bfill')

In [0]:
df=train.append(test,ignore_index=True)

In [0]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)   
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)   
    text = re.sub(r'www.[^ ]+', '', text)  
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

df['original_text'] = df['original_text'].apply(clean_text)

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import string
punctuation=string.punctuation
df['word_count']=df['original_text'].apply(lambda x: len(str(x).split(" ")))
df['char_count'] = df['original_text'].str.len()
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

df['avg_word'] = df['original_text'].apply(lambda x: avg_word(x))
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['stopwords'] = df['original_text'].apply(lambda x: len([x for x in x.split() if x in stop]))
df['numerics'] = df['original_text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df['upper'] = df['original_text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df['word_density'] = df['char_count'] / (df['word_count']+1)
df['punctuation_count'] = df['original_text'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation)))

In [0]:
j=[]
for i in df['original_text']:
  j.append(len(i))
df['len']=j

In [0]:
from textblob import TextBlob
df['polarity'] = df.apply(lambda x: TextBlob(x['original_text']).sentiment.polarity, axis=1)
df['subjectivity'] = df.apply(lambda x: TextBlob(x['original_text']).sentiment.subjectivity, axis=1)

In [13]:
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [0]:
'''
vader=SentimentIntensityAnalyzer()
v1=[]
v2=[]
v3=[]
v4=[]
for i in df['original_text']:
  j=vader.polarity_scores(i)
  j=list(j.values())
  v1.append(j[0])
  v2.append(j[1])
  v3.append(j[2])
  v4.append(j[3])
df['v1']=v1
df['v2']=v2
df['v3']=v3
df['v4']=v4
'''

"\nvader=SentimentIntensityAnalyzer()\nv1=[]\nv2=[]\nv3=[]1\nv4=[]\nfor i in df['original_text']:\n  j=vader.polarity_scores(i)\n  j=list(j.values())\n  v1.append(j[0])\n  v2.append(j[1])\n  v3.append(j[2])\n  v4.append(j[3])\ndf['v1']=v1\ndf['v2']=v2\ndf['v3']=v3\ndf['v4']=v4\n"

In [14]:
df.head(2)

,id,original_text,lang,retweet_count,original_author,sentiment_class,word_count,char_count,avg_word,stopwords,numerics,upper,word_density,punctuation_count,len,polarity,subjectivity
0,1.245025e+18,happy mothersday all you amazing mothers out t...,en,0,BeenXXPired,0.0,33,201,5.121212,14,0,0,5.911765,0,201,0.268056,0.677778
1,1.245759e+18,happy mothers day mum sorry can there bring yo...,en,1,FestiveFeeling,0.0,38,209,4.526316,10,0,0,5.358974,0,209,0.378571,0.782143


In [0]:
train = df[df['sentiment_class'].isnull()==False]
test = df[df['sentiment_class'].isnull()==True]

In [16]:
loc = np.append(train['original_author'].values, test['original_author'].values, axis=0)
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['original_author']=l.transform(train['original_author'])
test['original_author']=l.transform(test['original_author'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
loc = np.append(train['lang'].values, test['lang'].values, axis=0)
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['lang']=l.transform(train['lang'])
test['lang']=l.transform(test['lang'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
loc = np.append(train['retweet_count'].values, test['retweet_count'].values, axis=0)
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
train['retweet_count']=l.transform(train['retweet_count'])
test['retweet_count']=l.transform(test['retweet_count'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
train.head(2)

,id,original_text,lang,retweet_count,original_author,sentiment_class,word_count,char_count,avg_word,stopwords,numerics,upper,word_density,punctuation_count,len,polarity,subjectivity
0,1.245025e+18,happy mothersday all you amazing mothers out t...,307,25,190,0.0,33,201,5.121212,14,0,0,5.911765,0,201,0.268056,0.677778
1,1.245759e+18,happy mothers day mum sorry can there bring yo...,307,48,423,0.0,38,209,4.526316,10,0,0,5.358974,0,209,0.378571,0.782143


In [20]:
test.head(2)

,id,original_text,lang,retweet_count,original_author,sentiment_class,word_count,char_count,avg_word,stopwords,numerics,upper,word_density,punctuation_count,len,polarity,subjectivity
3235,1.246628e+18,yeah once cooked potatoes when was years old a...,307,25,631,NaN,37,223,5.054054,11,0,0,5.868421,0,223,0.150694,0.489583
3236,1.245898e+18,happy mother day all the mums step mums grandm...,307,25,1548,NaN,23,156,5.826087,5,0,0,6.500000,0,156,0.800000,1.000000


In [0]:
del test['sentiment_class']

In [0]:
del train['original_text']
del test['original_text']

In [23]:
train['sentiment_class'].value_counts()

 0.0    1701
-1.0     769
 1.0     765
Name: sentiment_class, dtype: int64

In [0]:
m=[]
for i in train['sentiment_class']:
    if i==0:
        m.append('A')
    elif i==1:
        m.append('B')
    else:
        m.append('C')


In [25]:
del train['sentiment_class']
train['sentiment_class']=m

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
train['sentiment_class'].value_counts()

A    1701
C     769
B     765
Name: sentiment_class, dtype: int64

#def rule

In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import matplotlib as mpl

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, learning_curve, ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix

In [0]:
X = train.drop(columns=['id','sentiment_class'])
y = train.sentiment_class

In [30]:
print('Percentage of each class in Target Variable \n')
print((train.sentiment_class.value_counts()/len(train))*100)

Percentage of each class in Target Variable 

A    52.581144
C    23.771252
B    23.647604
Name: sentiment_class, dtype: float64


In [31]:
rfc = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42, max_features=0.7 , bootstrap=False)
cross_val_score(rfc, X, y, cv=5, n_jobs=-1, verbose=1, scoring='f1_weighted').mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   16.8s finished


0.4151176509381983

In [32]:
y = train.sentiment_class
corr = pd.concat(objs=[X, y], axis=1).corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(5)

/usr/local/lib/python3.6/dist-packages/pandas/io/formats/style.py:1089: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
/usr/local/lib/python3.6/dist-packages/pandas/io/formats/style.py:1090: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


,lang,retweet_count,original_author,word_count,char_count,avg_word,stopwords,numerics,upper,word_density,punctuation_count,len,polarity,subjectivity
lang,1.00000,-0.46469,0.36968,-0.00289,-0.00187,-0.00080,-0.01608,nan,nan,-0.00046,nan,-0.00187,-0.01484,-0.01373
retweet_count,-0.46469,1.00000,-0.22230,0.00102,-0.00323,-0.01038,0.01194,nan,nan,-0.01092,nan,-0.00323,0.01611,0.00664
original_author,0.36968,-0.22230,1.00000,-0.00660,-0.00091,0.01479,-0.01452,nan,nan,0.01718,nan,-0.00091,-0.01777,0.00255
word_count,-0.00289,0.00102,-0.00660,1.00000,0.89913,-0.36975,0.77114,nan,nan,-0.28232,nan,0.89913,-0.33763,-0.30989
char_count,-0.00187,-0.00323,-0.00091,0.89913,1.00000,0.05512,0.53981,nan,nan,0.14840,nan,1.00000,-0.23424,-0.20488
avg_word,-0.00080,-0.01038,0.01479,-0.36975,0.05512,1.00000,-0.58320,nan,nan,0.99381,nan,0.05512,0.28149,0.28599
stopwords,-0.01608,0.01194,-0.01452,0.77114,0.53981,-0.58320,1.00000,nan,nan,-0.52771,nan,0.53981,-0.30382,-0.24917
numerics,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
upper,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
word_density,-0.00046,-0.01092,0.01718,-0.28232,0.14840,0.99381,-0.52771,nan,nan,1.00000,nan,0.14840,0.25374,0.25970


In [0]:
X_test = test.drop(columns=['id'])
concat = pd.concat(objs=[X, X_test], axis=0)

In [0]:
#concat.drop(columns=['numerics','upper','punctuation_count'
#                     ], inplace=True)

ss = StandardScaler()
concat = ss.fit_transform(concat)

X, X_test = concat[:len(train), :], concat[len(train):, :]

In [0]:
rfc = RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=42, max_features=0.9 , bootstrap=False)
cross_val_score(rfc, X, y, cv=45, n_jobs=-1, verbose=1, scoring='f1_weighted').mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   11.9s finished


0.414017782020411

In [0]:
nnc = MLPClassifier(hidden_layer_sizes=(10,5), max_iter=100,activation = 'logistic',solver='adam', 
                           random_state=42, batch_size=50)
cross_val_score(nnc, X, y, cv=5, n_jobs=-1, verbose=1, scoring='f1_weighted').mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    8.9s finished


0.3624010477350295

In [35]:
knc = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
cross_val_score(knc, X, y, cv=5, n_jobs=-1, verbose=1, scoring='f1_weighted').mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


0.4180324833036491

In [84]:
lgbmc = LGBMClassifier(random_state=42, n_estimators=410, colsample_bytree=1, 
                       max_depth=31, learning_rate=0.1, boosting_type='gbdt')
cross_val_score(lgbmc, X, y, cv=15, n_jobs=-1, verbose=1, scoring='f1_weighted').mean()
#0.4241955035198435 #400 #0.1 #15


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   22.6s finished


0.4226376190378294

In [118]:
xgbc = XGBClassifier(seed=7, n_jobs=-1, n_estimators=25, random_state=42, max_depth=31, learning_rate=0.1)
cross_val_score(xgbc, X, y, cv=15, n_jobs=-1, verbose=1, scoring='f1_weighted').mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   18.2s finished


0.4266809042540095

In [92]:
from sklearn.ensemble import VotingClassifier
ensemble = VotingClassifier(estimators=[#('rfc', rfc), 
                                        ('xgbc', xgbc),
                                        ('lgbmc', lgbmc)],
                                         voting='soft', n_jobs=-1)
cross_val_score(ensemble, X, y, cv=15, n_jobs=-1, verbose=1, scoring='f1_weighted').mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   29.7s finished


0.4227931313908444

In [0]:
lgbmc.fit(X, y)
j = lgbmc.predict(X_test)
sub = pd.DataFrame(data={'id' : test.id, 'sentiment_class' : j})
#sub.to_csv('mhack.csv', index=False)

In [0]:
m=[]
for i in sub['sentiment_class']:
    if i=='A':
        m.append('0')
    elif i=='B':
        m.append('1')
    else:
        m.append('-1')
sub['sentiment_class']=m

In [121]:
sub.head(2)

,id,sentiment_class
3235,1.246628e+18,0
3236,1.245898e+18,0


In [122]:
sub['sentiment_class'].value_counts()

0     908
-1    247
1     232
Name: sentiment_class, dtype: int64

In [0]:
sub.to_csv('ag.csv', index=False)

In [0]:
from google.colab import files
files.download('ag.csv')